# <center>**<font color=black>Sentiment Analysis on Product Reviews Using Bert**

> ***This notebook includes usage of Bert model. This is an example of sentiment analysis on product reviews in six languages: English,<br> Dutch, German, French, Spanish and Italian. The model predicts the sentiment of the review as a number of stars (between 1 and 5).***

### <font color=blue>Importing Packages

In [35]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

### <font color=blue>Instantiating The Model

> ***In this part, we take the pre-trained Bert model from the website 'Hugging Face' url given below:
https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment***

In [47]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### <font color=blue>Encoding and Calculating Sentiment

> ***In this part, we try to make encoding available for usage. It can be seen that encode function of tokenizer enables us to encode any sentence or paragraph according to pre-trained model weights. After that, if we give the encoded vector to the model, we can obtain some beneficial information.Then, we can get the result of the sentiment using argmax function of torch library***

In [15]:
tokens = tokenizer.encode('There was a situation about it. I dont know what but it was something terrible.', return_tensors='pt')

In [16]:
tokens[0]

tensor([  101, 10768, 10140,   143, 16894, 10935, 10197,   119,   151, 11930,
        16332, 11523, 10502, 10197, 10140, 19501, 50334,   119,   102])

In [17]:
tokenizer.decode(tokens[0])

'[CLS] there was a situation about it. i dont know what but it was something terrible. [SEP]'

In [19]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.8566,  1.8358,  0.6770, -1.3886, -2.5452]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [22]:
result.logits

tensor([[ 1.8566,  1.8358,  0.6770, -1.3886, -2.5452]],
       grad_fn=<AddmmBackward0>)

In [23]:
int(torch.argmax(result.logits)+1)

1

### <font color=blue>Collecting Reviews From a Website Using Web Scraping Technic

> ***In this part, we are trying to obtain the comments given to a restaurant from a website. We do this by using 'requests, BeautifulSoup4 and re' libraries. Requests package gives us the permission to be able to access the data of the website that we want to srap. After we get the permission, with BeautifulSoup4 library we can parse the website from its html codes. Then with re library, we can get only the desired part of the data, and with BeautifulSoup4 package we can search for this desired part in all page***

In [26]:
r = requests.get("https://www.yelp.com/biz/social-brew-cafe-pyrmont")
soup = BeautifulSoup(r.text,"html.parser")
regex = re.compile('.*comment*.')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [34]:
reviews[1]

"It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast roll was yum which did make it sing. So perhaps I just got a bad coffee but the food was good on my visit."

### <font color=blue>Loading Reviews into DataFrame and Score

In [36]:
df = pd.DataFrame(np.array(reviews), columns = ["reviews"])

In [38]:
df["reviews"].iloc[1]

"It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast roll was yum which did make it sing. So perhaps I just got a bad coffee but the food was good on my visit."

In [41]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits)+1)

In [42]:
sentiment_score(df["reviews"].iloc[1])

3

In [45]:
df["sentiment score"] = df["reviews"].apply(lambda x: sentiment_score(x[:1024]))
df.head(10)

,reviews,sentiment score
0,Great staff and food. Must try is the pan fri...,5
1,It was ok. The coffee wasn't the best but it w...,3
2,I came to Social brew cafe for brunch while ex...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I went here a little while ago- a beautiful mo...,1
5,We came for brunch twice in our week-long visi...,5
6,Ron & Jo are on the go down under and Wow! We...,5
7,Great coffee and vibe. That's all you need. C...,5
8,Great coffee and vibe. That's all you need. C...,4
9,Good coffee and toasts. Straight up and down -...,5
